## Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

## Q1. Downloading the data

In [1]:
import pandas as pd

In [ ]:
# pip install pyarrow

In [2]:
jan_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'
feb_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet'
df_jan = pd.read_parquet(jan_url)
df_feb = pd.read_parquet(feb_url)
#print(df.head())


In [4]:
# Read the data for January. How many columns are there?
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [5]:
jan_rows = df_jan.shape[0]
jan_rows

3066766

In [6]:
df_jan.shape

(3066766, 19)

In [7]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## Q2. Computing duration

In [8]:
# What's the standard deviation of the trips duration in January?
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)
df_jan['duration'].std()

42.59435124195458

## Q3. Dropping outliers

In [9]:
# What fraction of the records left after you dropped the outliers?
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
(df_jan.shape[0]/jan_rows)* 100

98.1220282212598

## Q4. One-hot encoding

In [ ]:
# pip install scikit-learn

In [13]:
# What's the dimensionality of this matrix (number of columns)?
from sklearn.feature_extraction import DictVectorizer

In [14]:
categorical = ['PULocationID', 'DOLocationID']
dict_features = df_jan[categorical].astype(str).to_dict(orient="records")

dv = DictVectorizer()
X_train = dv.fit_transform(dict_features)

In [15]:
X_train.shape

(3009173, 515)

In [16]:
print(f"Number of explicitly-stored values={X_train.getnnz()}")
print(f"Number of rows={X_train.shape[0]}")
print(f"Number of columns={X_train.shape[1]}")

Number of explicitly-stored values=6018346
Number of rows=3009173
Number of columns=515


## Q5. Training a model

In [17]:
# What's the RMSE on train?
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [18]:
target = 'duration'
y_train = df_jan[target].values

linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

LinearRegression()

In [19]:
y_pred = linear_reg.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

c:\Users\Daiyaan\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.649262223700304

## Q6. Evaluating the model

In [20]:
# What's the RMSE on validation?
df_feb['duration'] = (df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime).dt.total_seconds() / 60

In [21]:
df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]

In [22]:
valid_dicts = df_feb[categorical].astype("str").to_dict(orient="records")
X_valid = dv.transform(valid_dicts)

In [23]:
X_valid

<2855951x515 sparse matrix of type '<class 'numpy.float64'>'
	with 5711894 stored elements in Compressed Sparse Row format>

In [24]:
y_valid = df_feb["duration"]

In [27]:
y_pred = linear_reg.predict(X_valid)
mean_squared_error(y_valid, y_pred, squared=False)

c:\Users\Daiyaan\miniconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.811813608870425